# install packages

In [62]:
pip install nbformat

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# execute WalletClustering_AddressIdentification notebook

In [63]:
%run ./WalletClustering_AddressIdentification.ipynb # includes list creation of addresses
# methods & variables of notebook can be referenced

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [66]:
# only to remember lists
terrorAddressList # list of extracted flagged terror addresses
blacklistAddresses # list of exchange wallet addresses
addressesToCluster # list of flagged terror addresses without blacklist addresses

['12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8',
 '1348ThkNoDupq1bws95diMiL8haGs61K7M',
 '13Pcmh4dKJE8Aqrhq4ZZwmM1sbKFcMQEEV',
 '15K9Zj1AU2hjT3ebZMtWqDsMv3fFxTNwpf',
 '15soXrE3NJBMkkQhrccXonTT9bpjpPvE67',
 '164fawNZVwsR5SamAJypvCMtkMx4Xv1B3f',
 '179bzhS4FY7qLDza9YjuorhWyXVVYZu2YH',
 '17QAWGVpFV4gZ25NQug46e5mBho4uDP6MD',
 '17UUXDzPGkMwWrabhtk7YCha88tSoua2Vr',
 '19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW',
 '19XVEDZCGVMA9WCF1qUayxtnjUnyD7zDDQ',
 '1A7pDH1EdrkH9YZtsPnc8uzirBFnAN9Eay',
 '1BPf9qr7M5xUgNHUYtrQtEKvUKcyERzXao',
 '1C6hetVWVXZnS6P2BYBNu5Y1ZJ57JyXGac',
 '1DrhHEkv42JVwiDQNi28JFdSuiSGgPNXwP',
 '1EDcKCRypUTFoTZbxDWF9MBAT4W7XUGB32',
 '1EfmRn6Bp3cjrTBubaH8MzRRc2ikSjNGXw',
 '1EnX6BuJiGWydqXJT9BN5dSvfLg3QW4Mdz',
 '1EVTZmTMqZPMzGxsug9TXBtvPJZH8dXSCK',
 '1EYya5dfNvuYDwpeboGKBtkXzJcEHMCQXR',
 '1GALPyvUDDXqA6H2eHQ9Y1yidfQ6T1Drvn',
 '1GC2SjzCyCwxo1uxTi28oqn9L3mJj7bLPs',
 '1Gg25VzQkqCizXHNSNet4RoysLEe19su4s',
 '1JpSBaUwrZaEgmsYka7mzm9t3Z4syyaw7A',
 '1LhRW1msre1cFgT7fBY2BRrZ4ANMPwVj9u',
 '1Lm9BCDUKoBUk888DCXewM5

# cluster wallets

In [5]:
mihTemplate = '''
MATCH (:Address{address:"%s"})-[:SENDS]->(t:Transaction),
(walletMember:Address)-[:SENDS]->(t:Transaction)
//where t.outDegree <= 1
RETURN DISTINCT walletMember.address
'''

In [6]:
def multiInputClustering(address, walletAddresses):
    response = conn.query(mihTemplate % address, db='neo4j')
    newAddresses = [r[0] for r in response]
    newAddresses = [a for a in newAddresses if a not in walletAddresses]
    walletAddresses += newAddresses
    for a in newAddresses:
        walletAddresses += multiInputClustering(a, walletAddresses)
    return list(set(walletAddresses))

In [10]:
#same logic as Jochen's code; However, extremly large bitcoin wallets get excluded sofar because
#they take too long to be clustered. Example address: 13iQsrwBYdrLpnitG5EV79o3PeHjH8XUBc has more than 137k
#addresses in the wallet according to walletexplorer.com
#What should we do with them? I tried to solve it with multiprocessing but didn't help

def iterMultiInputClustering(address, walletIndex):
    walletString = "Terror-wallet ID"
    walletAddresses = []
    response = conn.query(mihTemplate % address, db='neo4j')
    newAddresses = [r[0] for r in response]
    walletAddresses = [newAddresses, walletString +str(walletIndex)]
    for walletAddress in walletAddresses[0]:
        if(len(walletAddresses[0])) >1000:
            return [address , "Too many results"];
        response = conn.query(mihTemplate % walletAddress, db='neo4j')
        newAddresses = [r[0] for r in response]
        newAddresses = [a for a in newAddresses if a not in walletAddresses[0]]
        walletAddresses[0] += newAddresses
    return walletAddresses

#walletIndex = 1
#wallets = iterMultiInputClustering("13iQsrwBYdrLpnitG5EV79o3PeHjH8XUBc", walletIndex)
#1EYya5dfNvuYDwpeboGKBtkXzJcEHMCQXR
#wallets


In [65]:
def callInputClustering():
    index = 0
    walletIndex = 1
    #["1EYya5dfNvuYDwpeboGKBtkXzJcEHMCQXR", "13iQsrwBYdrLpnitG5EV79o3PeHjH8XUBc", "1MMaU5nTrFdPZotfwdbv1wWnFjLCTFbpPY", "17QAWGVpFV4gZ25NQug46e5mBho4uDP6MD"]
    
    terrorCluster = []
    while index < len(addressesToCluster):
        terrorCluster +=iterMultiInputClustering(addressesToCluster[index], walletIndex)
        index +=1 
        walletIndex +=1
    return terrorCluster

In [13]:
# input = callInputClustering()
# 
# def flagAdditionalTerrorAdresses(input):
#     query = """
#     MATCH (a:Address {address: "%s"})
#     set a.isTerrorMultiInput = True,
#     a.terrorWallet = "%s"
#     Return a.address
#     """
#     indexWallets = 0
#     while indexWallets < len(input):
#         for address in input[indexWallets]:
#             walletID = input[indexWallets+1]
#             conn.query(query % (address, walletID), db='neo4j')
#         indexWallets += 2
# # flagAdditionalTerrorAdresses(input)
# 
# 